#### 3DCORE ABC SMC fitting example

output in /results/abc_runs

In [2]:
from matplotlib import pyplot as plt
import datetime
import logging
import numpy as np
import pickle

import py3dcore
import heliosat

from heliosat.coordinates import transform_pos
from heliosat.util import datetime_utc, datetime_utc_timestamp, datetime_to_string, sanitize_datetimes, string_to_datetime
from py3dcore.util import configure_logging, select_model

configure_logging(debug=False, logfile=None, verbose=True)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")

c0 = "xkcd:black"
c1 = "xkcd:magenta"
c2 = "xkcd:orange"
c3 = "xkcd:azure"

In [4]:
class SOLO_EXTRA_APRIL(heliosat.SOLO):
    
    def get_data_raw(self, range_start, range_end, data_key, **kwargs):
        
        logger = logging.getLogger(__name__)
        frame = kwargs.get("frame")
        data_key = self.resolve_data_key(data_key)
        range_start = sanitize_datetimes(range_start)
        range_end = sanitize_datetimes(range_end)
        frame = kwargs.get("frame", None)
        frame_cadence = kwargs.get("frame_cadence", None)
        force_download = kwargs.get("force_download", False)
        _data = pickle.load(open("/nas/helio/data/insitu_python/solo_2020_june_july_rtn.p", "rb"))
        time_all = np.array([_[0].replace(tzinfo=datetime.timezone.utc) for _ in _data])
        data_all = np.array([[_[1], _[2], _[3]] for _ in _data]).astype(np.float64)
        tfilter = (time_all > range_start) & (time_all < range_end)
        time_all = time_all[tfilter]
        data_all = data_all[tfilter]
        data_all =  transform_pos(time_all, data_all, "SOLO_SUN_RTN", frame)
        time_all = np.array([_.timestamp() for _ in time_all])
        
        return time_all, data_all


In [7]:
# ALL PARAMS
   
t_launch = datetime.datetime(2020, 6, 2, 2, tzinfo=datetime.timezone.utc)
t_s_solo = datetime.datetime(2020, 6, 7, 6, tzinfo=datetime.timezone.utc)
t_e_solo = datetime.datetime(2020, 6, 8, 12, tzinfo=datetime.timezone.utc)
t_solo = [
    datetime.datetime(2020, 6, 7, 12, tzinfo=datetime.timezone.utc),
    datetime.datetime(2020, 6, 7, 14, tzinfo=datetime.timezone.utc),
    datetime.datetime(2020, 6, 7, 16, tzinfo=datetime.timezone.utc),
    datetime.datetime(2020, 6, 7, 18, tzinfo=datetime.timezone.utc),
    datetime.datetime(2020, 6, 8, 1, tzinfo=datetime.timezone.utc),
    datetime.datetime(2020, 6, 8, 3, tzinfo=datetime.timezone.utc),
    datetime.datetime(2020, 6, 8, 5, tzinfo=datetime.timezone.utc),
    datetime.datetime(2020, 6, 8, 7, tzinfo=datetime.timezone.utc),
    datetime.datetime(2020, 6, 8, 9, tzinfo=datetime.timezone.utc),
    datetime.datetime(2020, 6, 8, 11, tzinfo=datetime.timezone.utc),
]
# FRONT PARAMS
#t_s_solo = datetime.datetime(2020, 6, 7, 6, tzinfo=datetime.timezone.utc)
#t_e_solo = datetime.datetime(2020, 6, 8, tzinfo=datetime.timezone.utc)
#t_solo = [
#    datetime.datetime(2020, 6, 7, 12, tzinfo=datetime.timezone.utc),
#    datetime.datetime(2020, 6, 7, 14, tzinfo=datetime.timezone.utc),
#    datetime.datetime(2020, 6, 7, 16, tzinfo=datetime.timezone.utc),
#    datetime.datetime(2020, 6, 7, 18, tzinfo=datetime.timezone.utc),
#]
# BACK PARAMS
#t_s_solo = datetime.datetime(2020, 6, 8, tzinfo=datetime.timezone.utc)
#t_e_solo = datetime.datetime(2020, 6, 8, 12, tzinfo=datetime.timezone.utc)
#t_solo = [
#    datetime.datetime(2020, 6, 8, 1, tzinfo=datetime.timezone.utc),
#    datetime.datetime(2020, 6, 8, 3, tzinfo=datetime.timezone.utc),
#    datetime.datetime(2020, 6, 8, 5, tzinfo=datetime.timezone.utc),
#    datetime.datetime(2020, 6, 8, 7, tzinfo=datetime.timezone.utc),
#    datetime.datetime(2020, 6, 8, 9, tzinfo=datetime.timezone.utc),
#    datetime.datetime(2020, 6, 8, 11, tzinfo=datetime.timezone.utc),
#]

abcsmc = py3dcore.fitting.ABCSMC_v1()
abcsmc.init(t_launch, select_model("TTGHv2"), seed=42)
abcsmc.add_observer(SOLO_EXTRA_APRIL, t_solo, t_s_solo, t_e_solo, dt=8, dd=1, dti=0)
abcsmc.param_minmax("cme_longitude", 230, 320)

#abcsmc.param_fix("cme_width_ratio", 1)
#abcsmc.param_minmax("cme_longitude", 140, 320)
#abcsmc.param_minmax("cme_aspect_ratio", None , None, "log-uniform")
#abcsmc.param_minmax("cme_diameter_1au", None, None, "log-uniform")

abcsmc.param_minmax("noise", 0, 10)

abcsmc.run(12, eps_quantile=.25, jobs=128,
           kernel_mode="lcm", output="results/abc_runs/june_full", runs=16, 
           sub_iter_max=64, workers=128, particles=4 * 1024, summary="rmse")


2021-01-19 14:27:36,616 - py3dcore.fitting.methods.abcsmc_v1 - starting abc algorithm
2021-01-19 14:27:36,616 - py3dcore.fitting.methods.abcsmc_v1 - starting abc algorithm
2021-01-19 14:27:36,617 - py3dcore.fitting.methods.abcsmc_v1 - starting iteration 0
2021-01-19 14:27:36,617 - py3dcore.fitting.methods.abcsmc_v1 - starting iteration 0
2021-01-19 14:27:36,620 - py3dcore.fitting.methods.abcsmc_v1 - obs 0 marker offset set to 8.0 hours
2021-01-19 14:27:36,620 - py3dcore.fitting.methods.abcsmc_v1 - obs 0 marker offset set to 8.0 hours
2021-01-19 14:27:36,626 - py3dcore.fitting.methods.abcsmc_v1 - initial eps_0 = 20.58nT
2021-01-19 14:27:36,626 - py3dcore.fitting.methods.abcsmc_v1 - initial eps_0 = 20.58nT


IndexError: index 12 is out of bounds for axis 0 with size 12